In [15]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from tqdm import tqdm
folder = Path.cwd().parent
from pydub import AudioSegment
from pydub.playback import play

number = '0010'

In [16]:
main_path = folder / '_storage' / 'main' / number

print('1')

pickle_in = open(main_path / 'network.pickle'  , 'rb' )
network = pickle.load(pickle_in)
pickle_in.close()

print('2')

pickle_in2 = open(main_path / 'changes.pickle' , 'rb' )
RK4 = pickle.load(pickle_in2)
pickle_in2.close()

print('3')

pickle_in3 = open(main_path / 'vessels.pickle' , 'rb' )
vessels_alltime = pickle.load(pickle_in3)
pickle_in3.close()

print('4')

1
2
3
4


In [17]:
def time_step_display(i):
    if i != 0:
        print(network.iloc[[i-1,i,i+1],:])
    else:
        print(network.iloc[[i,i+1],:])
    # print(pd.DataFrame(network.iloc[i-1,:]).transpose)
    # print(network.iloc[i,:])
    # print(network.iloc[i+1,:])

    print(vessels_alltime[i])

    print(RK4[i])

def combine_network_changes(network,RK4):
    network['dphidt'] = np.nan
    network['dApdt'] = np.nan
    network['dDpdt'] = np.nan
    network['dAndt'] = np.nan
    network['dDndt'] = np.nan
    network['dcdt'] = np.nan
    for i in tqdm(range(len(network)-5)):
        network.at[i,'dphidt'] = RK4[i][4]
        network.at[i,'dApdt'] = RK4[i][0]
        network.at[i,'dDpdt'] = RK4[i][1]
        network.at[i,'dAndt'] = RK4[i][2]
        network.at[i,'dDndt'] = RK4[i][3]
        network.at[i,'dcdt'] = RK4[i][5]

    # print(network)

    network.to_csv(main_path / 'combined_network.csv')
    
def combine_vessels_changes(vessels_alltime,RK4):
    for i in tqdm(range(len(vessels_alltime)-5)):
        vessels_alltime[i]['dptdt'] = RK4[i][6]['dptdt']
        vessels_alltime[i]['dSoutdt'] = RK4[i][7]['dSoutdt']
        vessels_alltime[i]['dSindt'] = RK4[i][8]['dSindt']

    save = pd.concat(vessels_alltime)

    save.to_csv(main_path / 'combined_vessels.csv')
    
    # pickle_out = open(main_path / 'combined_vessels.pickle','wb')
    # pickle.dump(save,pickle_out)
    # pickle_out.close()

In [18]:
combine_network_changes(network,RK4)
out = combine_vessels_changes(vessels_alltime,RK4)



# sound = AudioSegment.from_mp3('minecraft.mp3')
# play(sound)

100%|██████████| 4/4 [00:00<00:00, 522.69it/s]
/Users/Debs/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [19]:
type(network)

pandas.core.frame.DataFrame